In [1]:
import os, glob
import soundfile as sf
import numpy as np
import pandas as pd
import h5py
import tqdm
import IPython
import fairseq
import torch
from matplotlib import pyplot as plt
from sklearn.manifold import TSNE
from sklearn import svm
from sklearn import metrics
import seaborn as sns
import utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from fairseq.modules import Fp32LayerNorm,  TransposeLast
import pytorch_lightning as pl
import torch.nn as nn
import utils

2021-11-24 21:36:45 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


# Convert wav to mel

In [22]:
!rm -rf ./data/TIMIT/train/mel.h5

In [26]:
h5out = h5py.File("./data/TIMIT/test/mel.h5", 'w')

In [27]:
df = pd.read_csv("./data/TIMIT_test.csv")
for i, row in df.iterrows():
    s, sr = sf.read(row['wav_path'], dtype='float32')
    mel = utils.convert_to_mel(s, sr=16000, nfft=400, win_length=400, hop_length=320, power=2, n_mels=64, normalized=False)
    h5out.create_dataset(row['wav_id'], data=mel[0].T)

In [28]:
h5out.close()

In [15]:
!echo melSpectrogram.h5 sr=16000, nfft=1024, win_length=720, hop_length=320, power=2, n_mels=128, normalized=False >> info.txt

In [30]:
df = pd.read_csv("./data/TIMIT_train.csv")

In [31]:
df

,wav_id,wav_path,dialect,sentence,speaker,gender
0,DR1_FCJF0_SA1,./data/data/TRAIN/DR1/FCJF0/SA1.WAV.wav,DR1,SA1,FCJF0,F
1,DR1_FCJF0_SA2,./data/data/TRAIN/DR1/FCJF0/SA2.WAV.wav,DR1,SA2,FCJF0,F
2,DR1_FCJF0_SI1027,./data/data/TRAIN/DR1/FCJF0/SI1027.WAV.wav,DR1,SI1027,FCJF0,F
3,DR1_FCJF0_SI1657,./data/data/TRAIN/DR1/FCJF0/SI1657.WAV.wav,DR1,SI1657,FCJF0,F
4,DR1_FCJF0_SI648,./data/data/TRAIN/DR1/FCJF0/SI648.WAV.wav,DR1,SI648,FCJF0,F
...,...,...,...,...,...,...
4615,DR8_MTCS0_SX172,./data/data/TRAIN/DR8/MTCS0/SX172.WAV.wav,DR8,SX172,MTCS0,M
4616,DR8_MTCS0_SX262,./data/data/TRAIN/DR8/MTCS0/SX262.WAV.wav,DR8,SX262,MTCS0,M
4617,DR8_MTCS0_SX352,./data/data/TRAIN/DR8/MTCS0/SX352.WAV.wav,DR8,SX352,MTCS0,M
4618,DR8_MTCS0_SX442,./data/data/TRAIN/DR8/MTCS0/SX442.WAV.wav,DR8,SX442,MTCS0,M


In [54]:
h5old = h5py.File("./outputs/extracted_features/wav2vec2_small-random_init//TIMIT_train_bk.h5", 'r')
h5new = h5py.File("./outputs/extracted_features/wav2vec2_small-random_init//TIMIT_train.h5", 'w')

In [55]:
for key in h5old.keys():
    new_key = key.split('.')[0] + '-' + key.split('-')[1]
    h5new.create_dataset(new_key, data=h5old[key][:])
h5old.close()
h5new.close()

In [53]:
!mv ./outputs/extracted_features/wav2vec2_small-random_init//TIMIT_train.h5 ./outputs/extracted_features/wav2vec2_small-random_init//TIMIT_train_bk.h5

In [48]:
h5old.keys()

<KeysViewHDF5 ['DR1_FCJF0_SA1.WAV-cnn_output', 'DR1_FCJF0_SA1.WAV-context_vector', 'DR1_FCJF0_SA1.WAV-encoder_output', 'DR1_FCJF0_SA1.WAV-projected_vq', 'DR1_FCJF0_SA1.WAV-vq', 'DR1_FCJF0_SA2.WAV-cnn_output', 'DR1_FCJF0_SA2.WAV-context_vector', 'DR1_FCJF0_SA2.WAV-encoder_output', 'DR1_FCJF0_SA2.WAV-projected_vq', 'DR1_FCJF0_SA2.WAV-vq', 'DR1_FCJF0_SI1027.WAV-cnn_output', 'DR1_FCJF0_SI1027.WAV-context_vector', 'DR1_FCJF0_SI1027.WAV-encoder_output', 'DR1_FCJF0_SI1027.WAV-projected_vq', 'DR1_FCJF0_SI1027.WAV-vq', 'DR1_FCJF0_SI1657.WAV-cnn_output', 'DR1_FCJF0_SI1657.WAV-context_vector', 'DR1_FCJF0_SI1657.WAV-encoder_output', 'DR1_FCJF0_SI1657.WAV-projected_vq', 'DR1_FCJF0_SI1657.WAV-vq', 'DR1_FCJF0_SI648.WAV-cnn_output', 'DR1_FCJF0_SI648.WAV-context_vector', 'DR1_FCJF0_SI648.WAV-encoder_output', 'DR1_FCJF0_SI648.WAV-projected_vq', 'DR1_FCJF0_SI648.WAV-vq', 'DR1_FCJF0_SX127.WAV-cnn_output', 'DR1_FCJF0_SX127.WAV-context_vector', 'DR1_FCJF0_SX127.WAV-encoder_output', 'DR1_FCJF0_SX127.WAV-proj

In [49]:
h5old.close()

In [59]:
a = [1,2]
np.random.choice(a, 10)

array([2, 2, 1, 2, 2, 1, 2, 1, 2, 2])